# Module 4: Anomaly Detection
## Practice: Outlier Reduction for Linear Regression
In this session, we'll be fitting a `LinearRegression` model on the `boston` dataset included in `scikit-learn`.  

Having already worked with this dataset,
you may remember it as a simple yet broadly representative linear regression problem.

## Getting started - imports

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error

np.random.seed(10)

## Loading dataset
First order of business is to load in the dataset.

In [2]:
# Load boston housing dataset 
boston = load_boston()
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [3]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [4]:
type(boston.data)

numpy.ndarray

In [5]:
# Split into X and y sets   #P4001

X = boston.data
y = boston.target

# Print out some basic shape data on the arrays
print("X, y shape:", X.shape, y.shape)

X, y shape: (506, 13) (506,)


In [6]:
# perform train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)  #P4002

# verify split shapes and contents
print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)

X_train.shape:  (404, 13)
y_train.shape:  (404,)
X_test.shape:  (102, 13)
y_test.shape:  (102,)


Run cross validation (5-fold) on a linear ridge model.

In [8]:
naive_model = LinearRegression() #P4003
scores = cross_val_score(estimator = naive_model, X = X_train, y = y_train)
print("Scores: ", scores)
print("Mean score (5 folds): ", np.mean(scores))

Scores:  [0.66285345 0.78752977 0.77137863 0.73129637 0.63939938]
Mean score (5 folds):  0.718491518543017


Fit this model on the training dataset.

In [10]:
# Fit a model with all the training data #P4004
naive_model.fit(X_train, y_train)


LinearRegression()

Make some predictions from testing dataset and print mean absolute measure.

In [11]:
naive_predictions = naive_model.predict(X_test) #P4005
print(X_test.shape, naive_predictions.shape)

mean_absolute_error(y_test, naive_predictions)


(102, 13) (102,)


4.061419182954701


## What methods are available to us for outlier reduction?
We could try `KMeans` or an `EllipticEnvelope` again, but we're going to explore a few more options. 

In [13]:
from sklearn.ensemble import IsolationForest

# Construct IsolationForest 
iso_forest = IsolationForest(contamination=0.05).fit(X_train, y_train)

# Get labels from classifier and cull outliers #P4006
iso_outliers = iso_forest.predict(X_train) == -1
print(f"Num of outliers = {np.sum(iso_outliers)}")
X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]


Num of outliers = 21


In [14]:
# develop a liner regression model without outliers 

iso_model = LinearRegression()
iso_model.fit(X_iso, y_iso)

# Cross validate the new model
iso_scores = cross_val_score(estimator = iso_model, X = X_iso, y = y_iso)
print(iso_scores)
print("Mean CV score w/ IsolationForest:", np.mean(iso_scores))

iso_predictions = iso_model.predict(X_test)
mean_absolute_error(y_test, iso_predictions)


[0.64998117 0.79432671 0.76187    0.76921453 0.60922113]
Mean CV score w/ IsolationForest: 0.7169227069812166


4.044947300109876

## Alternatives to IsolationForest: OneClassSVM
This means it's time to try something else.  
The code below will look very similar to the above, but using `OneClassSVM` in place of the `IsolationForest`:

In [18]:
from sklearn.svm import OneClassSVM

# Construct OneClassSVM (kernel='rbf') and fit to full dataset
svm = OneClassSVM(kernel='rbf').fit(X_train, y_train)

# Get labels from classifier and cull outliers #P4007
svm_outliers = svm.predict(X_train) == -1
print(f"Num of outliers = {np.sum(iso_outliers)}")
X_svm = X_train[~svm_outliers]
y_svm = y_train[~svm_outliers]


Num of outliers = 21


In [19]:
# develop a linear regression model without outliers 

svm_model = LinearRegression().fit(X_svm, y_svm)

# Cross validate the new model
svm_scores = cross_val_score(estimator = svm_model, X = X_svm, y = y_svm)
print(svm_scores)
print("Mean CV score w/ OneClassSVM:", np.mean(svm_scores))

# Make predictions with the fitted model
svm_predictions = svm_model.predict(X_test)

mean_absolute_error(y_test, svm_predictions)


[ 0.66400765  0.7291245   0.5624935   0.68477104 -0.48927007]
Mean CV score w/ OneClassSVM: 0.43022532584013806


3.9794908013350256

## Summary Analysis

Of the anomaly detection algorithms used, which had the highest marginal performance. 

## Addtional Tasks
Vary various parameters and performance measures for the above practice and see the performance.